## Overfitting 실습
모델이 Training set에 대해 과적합되어, 새로운 데이터인 Test Set에는 적용되지 못하는 현상을 관찰해보겠다.

####Imports

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# For reproductibitily
torch.manual_seed(1)

#### Training Set과 Testing Set 나누어서 선언해줌

In [0]:
x_train = torch.FloatTensor([[1, 2, 1],
                            [1, 3, 2],
                            [1, 3, 4],
                            [1, 5, 5],
                            [1, 7, 5],
                            [1, 2, 5],
                            [1, 6, 6],
                            [1, 7, 7]])
# |x_train| = (m, 3)

y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])
# |y_train| = (m,  ) One Hot Vector의 인덱스들을 가지고 있음 

x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
# |x_test| = (m', 3)    x_train과 feature 수가 같아야 하므로 똑같이 3

y_test = torch.LongTensor([2, 2, 2])
# |y_test| = (m',  )


####모델 설정. 
이전에 배웠던 Softmax Regression용 모듈을 적용한다.

In [0]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 3)
    
  def forward(self, x):
    return self.linear(x)

model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 0.1)

####  관찰하기
Training Set에 대해 위 모듈을 적용해 모델을 학습시키는 함수를 만든다.

In [0]:
# Training data에 대해 Seftmax Regression을 20회에 걸쳐 수행하고 결과 출력하는 함수
def train(model, optimizer, x_train, y_train) : 
  nb_epochs = 20;
  for epoch in range (nb_epochs):

    # H(x) 계산
    prediction = model(x_train)   # |prediction| = |x_train| = (m, 3)


    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 계산

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('E: {:4d}/{}  Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))


Test를 통해 Accuracy를 구하는 함수도 만들어준다.

In [0]:
def test(model, opt, x_test, y_test):
  prediction = model(x_test)                  # model을 test set에 통과시킴. |x_test| = (m',3)
  predicted_classes = prediction.max(1)[1]    # |prediction| = (m',3)
  # max 함수는 입력된 dimention의 방향에 맞는 최대값을 구해주는 함수였던 것 같다..
  correct_count = (predicted_classes == y_test).sum().item()
  cost = F.cross_entropy(prediction, y_test)  # 정답과의 cross entropy를 구한다.

  print('Accuracy: {}%   Cost: {:.6f}'.format(correct_count / len(y_test) * 100, cost.item()))

함수를 호출해 작동시키고, 결과를 지켜 본다.

In [0]:
train(model, optimizer, x_train, y_train)
test(model, optimizer, x_test, y_test)

E:    0/20  Cost: 2.203667
E:    1/20  Cost: 1.199645
E:    2/20  Cost: 1.142985
E:    3/20  Cost: 1.117769
E:    4/20  Cost: 1.100901
E:    5/20  Cost: 1.089523
E:    6/20  Cost: 1.079872
E:    7/20  Cost: 1.071320
E:    8/20  Cost: 1.063325
E:    9/20  Cost: 1.055720
E:   10/20  Cost: 1.048378
E:   11/20  Cost: 1.041245
E:   12/20  Cost: 1.034285
E:   13/20  Cost: 1.027478
E:   14/20  Cost: 1.020813
E:   15/20  Cost: 1.014279
E:   16/20  Cost: 1.007872
E:   17/20  Cost: 1.001586
E:   18/20  Cost: 0.995419
E:   19/20  Cost: 0.989365
Accuracy: 0.0%   Cost: 1.425844


위 결과에서 Trainig Set에 대한 Cost는 점차 낮아지는 것을 관찰 할 수 있으나,
Test Set에 대한 Cost는 외려 다소 큰 것을 볼 수 있다.
20 Epoch에 도달했을 지점엔 이미 overfitting이 진행되었다는 걸 알 수 있다. 

## Learning Rate

이번에는 Gradient Descent에서 learning rate을 너무 **크게** 잡았을 때 overshooting 하는 모습을 관찰해보겠다.

lr을 e의 5승(대략 148.4)으로 잡았다.

In [0]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr = 1e5)
train(model, optimizer, x_train, y_train)

E:    0/20  Cost: 1.280268
E:    1/20  Cost: 976950.750000
E:    2/20  Cost: 1279135.000000
E:    3/20  Cost: 1198379.125000
E:    4/20  Cost: 1098825.625000
E:    5/20  Cost: 1968197.625000
E:    6/20  Cost: 284763.125000
E:    7/20  Cost: 1532260.000000
E:    8/20  Cost: 1651504.250000
E:    9/20  Cost: 521878.437500
E:   10/20  Cost: 1397263.125000
E:   11/20  Cost: 750986.250000
E:   12/20  Cost: 918691.750000
E:   13/20  Cost: 1487888.125000
E:   14/20  Cost: 1582260.125000
E:   15/20  Cost: 685818.000000
E:   16/20  Cost: 1140048.750000
E:   17/20  Cost: 940566.750000
E:   18/20  Cost: 931638.125000
E:   19/20  Cost: 1971322.625000


이번에는 Learning rate이 너무 작을 때 cost 값이 잘 줄어들지 않는 것을 관찰하겠다.

In [0]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr = 1e-10)
train(model, optimizer, x_train, y_train)

E:    0/20  Cost: 3.187324
E:    1/20  Cost: 3.187324
E:    2/20  Cost: 3.187324
E:    3/20  Cost: 3.187324
E:    4/20  Cost: 3.187324
E:    5/20  Cost: 3.187324
E:    6/20  Cost: 3.187324
E:    7/20  Cost: 3.187324
E:    8/20  Cost: 3.187324
E:    9/20  Cost: 3.187324
E:   10/20  Cost: 3.187324
E:   11/20  Cost: 3.187324
E:   12/20  Cost: 3.187324
E:   13/20  Cost: 3.187324
E:   14/20  Cost: 3.187324
E:   15/20  Cost: 3.187324
E:   16/20  Cost: 3.187324
E:   17/20  Cost: 3.187324
E:   18/20  Cost: 3.187324
E:   19/20  Cost: 3.187324


아무튼 이렇게 학습이 발산을 하거나, 너무 더디게 진행될 경우 lr을 조절해주는 노하우가 필요하다. 

## Data Preprocessing

데이터 전처리(선처리)

In [0]:
x_train  = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train  = torch.FloatTensor([[152],[185],[180], [196], [142]])

Mean Square Error 값을 쓰게 될 것임.

아래 코드는 x_train 벡터들을 Normalization 시키는 과정이다.

방법 - Standardization; 정규화

$$ 
x'_j = \frac{x_j - \mu_j}{\sigma_j}
$$

In [0]:
mu = x_train.mean(dim=0)
sigma = x_train.std(dim=0)
norm_x_train = (x_train - mu)/sigma

print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


정규 분포를 따르는 training set을 만들어 주었다.


그 이후엔 데이터에 맞게 regression 모델을 짜주겠다.

In [0]:
class MultivariableLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

In [0]:
model = MultivariableLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr = 1e-1)

Training 함수 짜기

In [0]:
# Training 

def train(model, optimizer, x_train, y_train):
  nb_epochs = 20
  for epoch in range(nb_epochs):
    
    # H(x) 계산
    prediction = model(x_train)          
    # |x_train| = (m, 3)
    # |prediction| = (m, 1)

    # Cost는 MSE로 계산
    cost = F.mse_loss(prediction, y_train)
    # cost를 토대로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('E: {:4d} / {}   Cost: {:.6f} '.format(epoch, nb_epochs, cost.item()))

In [0]:
print("*** Training with Preprocessed Data")
train(model, optimizer, norm_x_train, y_train)

print("*** Without data Preprocessing... ")
model = MultivariableLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr = 1e-1)
train(model, optimizer, x_train, y_train)

*** Training with Preprocessed Data
E:    0 / 20   Cost: 29615.740234 
E:    1 / 20   Cost: 18803.878906 
E:    2 / 20   Cost: 11991.029297 
E:    3 / 20   Cost: 7661.868164 
E:    4 / 20   Cost: 4900.236816 
E:    5 / 20   Cost: 3135.413574 
E:    6 / 20   Cost: 2006.681396 
E:    7 / 20   Cost: 1284.504639 
E:    8 / 20   Cost: 822.366089 
E:    9 / 20   Cost: 526.605774 
E:   10 / 20   Cost: 337.314728 
E:   11 / 20   Cost: 216.160767 
E:   12 / 20   Cost: 138.613449 
E:   13 / 20   Cost: 88.974701 
E:   14 / 20   Cost: 57.197395 
E:   15 / 20   Cost: 36.851803 
E:   16 / 20   Cost: 23.822887 
E:   17 / 20   Cost: 15.476827 
E:   18 / 20   Cost: 10.128090 
E:   19 / 20   Cost: 6.697937 
*** Without data Preprocessing... 
E:    0 / 20   Cost: 57621.976562 
E:    1 / 20   Cost: 1115690762240.000000 
E:    2 / 20   Cost: 21603378185380036608.000000 
E:    3 / 20   Cost: 418311281336289389130547200.000000 
E:    4 / 20   Cost: 8099858389234773491298680935809024.000000 
E:    5 / 20   Co

normalization이 필요한 데이터셋은 어떤 걸까?
\|y_train\|이 (m, 2)와 같이 2차원 prediction 구조를 가질 때, 두 colomn 사이의 수의 스케일이 서로 다르다면 training 모델은 더 큰 값에만 집중을 하게 된다.

전처리를 수행하게 되면 똑같은 범위의 값으로 바뀔 것이고, 모델은 두가지 데이터에 대해 공정하게 학습을 할 수 있을 것이다.